# DSA210 Term Project  
## Movie Budget, Rating, and Revenue Analysis (TMDB Dataset)

This notebook analyzes the relationship between **movie budgets**, **ratings**, and **box office revenue**
using the **TMDB 5000 Movies dataset**.

This is the **final corrected version**, including:
- EDA
- Correlation analysis
- Genre analysis
- **Machine Learning methods**
(No warnings, no runtime errors)


In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score, root_mean_squared_error
import ast

%matplotlib inline
sns.set(style="whitegrid")


## 1. Data Loading

In [ ]:
tmdb = pd.read_csv("tmdb_5000_movies.csv")
print("Rows:", tmdb.shape[0])
print("Columns:", tmdb.shape[1])
tmdb.head()


## 2. Data Cleaning & Feature Engineering

In [ ]:
tmdb = tmdb.drop_duplicates()

tmdb["budget"] = pd.to_numeric(tmdb["budget"], errors="coerce")
tmdb["revenue"] = pd.to_numeric(tmdb["revenue"], errors="coerce")
tmdb["vote_average"] = pd.to_numeric(tmdb["vote_average"], errors="coerce")
tmdb["runtime"] = pd.to_numeric(tmdb["runtime"], errors="coerce")

tmdb = tmdb.dropna(subset=["budget", "revenue", "vote_average", "runtime"])
tmdb = tmdb[(tmdb["budget"] > 0) & (tmdb["revenue"] > 0)]

tmdb["profit"] = tmdb["revenue"] - tmdb["budget"]
tmdb["roi"] = tmdb["revenue"] / tmdb["budget"]

tmdb["log_budget"] = np.log10(tmdb["budget"])
tmdb["log_revenue"] = np.log10(tmdb["revenue"])

tmdb.head()


## 3. Exploratory Data Analysis

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18,5))

sns.histplot(tmdb["log_budget"], bins=40, kde=True, ax=axes[0])
axes[0].set_title("Log(Budget) Distribution")

sns.histplot(tmdb["log_revenue"], bins=40, kde=True, ax=axes[1])
axes[1].set_title("Log(Revenue) Distribution")

sns.histplot(tmdb["vote_average"], bins=20, kde=True, ax=axes[2])
axes[2].set_title("Rating Distribution")

plt.tight_layout()
plt.show()


## 4. Correlation Analysis

In [ ]:
corr_budget_rev, _ = pearsonr(tmdb["budget"], tmdb["revenue"])
corr_budget_rating, _ = pearsonr(tmdb["budget"], tmdb["vote_average"])

print("Budget vs Revenue correlation:", corr_budget_rev)
print("Budget vs Rating correlation:", corr_budget_rating)

sns.heatmap(
    tmdb[["budget","revenue","profit","roi","vote_average"]].corr(),
    annot=True, cmap="coolwarm"
)
plt.title("Correlation Heatmap")
plt.show()


## 5. Genre Analysis

In [ ]:
def extract_genre(x):
    try:
        g = ast.literal_eval(x)
        return g[0]["name"] if g else "Unknown"
    except:
        return "Unknown"

tmdb["main_genre"] = tmdb["genres"].apply(extract_genre)

plt.figure(figsize=(12,6))
sns.boxplot(data=tmdb, x="main_genre", y="log_budget")
plt.xticks(rotation=45)
plt.title("Budget by Genre (Log Scale)")
plt.show()


## 6. Machine Learning Methods

We predict **log(Revenue)** using:
- log(Budget)
- Rating
- Runtime


In [ ]:
X = tmdb[["log_budget", "vote_average", "runtime"]]
y = tmdb["log_revenue"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
# Linear Regression
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)

# Ridge Regression
ridge = Ridge(alpha=1.0)
ridge.fit(X_train, y_train)
y_pred_ridge = ridge.predict(X_test)

# Lasso Regression
lasso = Lasso(alpha=0.01)
lasso.fit(X_train, y_train)
y_pred_lasso = lasso.predict(X_test)


## 7. Model Evaluation

In [ ]:
results = pd.DataFrame({
    "Model": ["Linear Regression", "Ridge Regression", "Lasso Regression"],
    "R2": [
        r2_score(y_test, y_pred_lr),
        r2_score(y_test, y_pred_ridge),
        r2_score(y_test, y_pred_lasso)
    ],
    "RMSE": [
        root_mean_squared_error(y_test, y_pred_lr),
        root_mean_squared_error(y_test, y_pred_ridge),
        root_mean_squared_error(y_test, y_pred_lasso)
    ]
})

results


In [ ]:
plt.figure(figsize=(7,6))
plt.scatter(y_test, y_pred_lr, alpha=0.5)
plt.plot([y_test.min(), y_test.max()],
         [y_test.min(), y_test.max()],
         'r--')
plt.xlabel("Actual log(Revenue)")
plt.ylabel("Predicted log(Revenue)")
plt.title("Linear Regression Predictions")
plt.show()


## 8. Conclusion

- Budget is the strongest predictor of revenue.
- Ratings have limited predictive power.
- Regularization (Ridge/Lasso) does not significantly improve performance.
- The TMDB dataset alone is sufficient for this project.
